In [1]:
import re
import string


chars = sorted((string.ascii_lowercase+string.punctuation+string.digits+" "))
n_vocab = len(chars)
data_map = dict((c, i) for i, c in enumerate(chars))
print(data_map) 


{'i': 47, 'b': 40, 'd': 42, '?': 31, 'r': 56, '{': 65, '$': 4, ';': 27, '`': 38, 'h': 46, ']': 35, 'k': 49, '"': 2, '~': 68, 'o': 53, '|': 66, '9': 25, 'x': 62, ')': 9, ',': 12, 'p': 54, 'j': 48, '/': 15, 'l': 50, '@': 32, 'e': 43, '6': 22, ' ': 0, '3': 19, '\\': 34, '%': 5, '*': 10, "'": 7, '^': 36, '.': 14, '_': 37, ':': 26, '4': 20, '!': 1, '-': 13, 'y': 63, '[': 33, 'w': 61, 'v': 60, '1': 17, 'f': 44, 'c': 41, '+': 11, '&': 6, '>': 30, '<': 28, 'a': 39, 'g': 45, '#': 3, 'q': 55, '5': 21, '7': 23, 'm': 51, 't': 58, 'u': 59, 's': 57, '(': 8, '}': 67, 'n': 52, '=': 29, '2': 18, '8': 24, '0': 16, 'z': 64}


In [2]:

with open('normalTrafficTraining.txt') as f:
    arr = f.readlines()
    content = ''.join(arr)
    items = re.findall("^GET.*|POST.*$", content, re.MULTILINE)
    dataset = []
    for item in items:
        temp_data = item.replace(" HTTP/1.1","").lower()
        dataset.append(temp_data)
print(dataset[:20])

    

['get http://localhost:8080/tienda1/index.jsp', 'get http://localhost:8080/tienda1/publico/anadir.jsp?id=3&nombre=vino+rioja&precio=100&cantidad=55&b1=a%f1adir+al+carrito', 'post http://localhost:8080/tienda1/publico/anadir.jsp', 'get http://localhost:8080/tienda1/publico/autenticar.jsp?modo=entrar&login=choong&pwd=d1se3ci%f3n&remember=off&b1=entrar', 'post http://localhost:8080/tienda1/publico/autenticar.jsp', 'get http://localhost:8080/tienda1/publico/caracteristicas.jsp?id=2', 'post http://localhost:8080/tienda1/publico/caracteristicas.jsp', 'get http://localhost:8080/tienda1/publico/carrito.jsp', 'get http://localhost:8080/tienda1/publico/entrar.jsp?errormsg=credenciales+incorrectas', 'post http://localhost:8080/tienda1/publico/entrar.jsp', 'get http://localhost:8080/tienda1/publico/miembros.jsp', 'get http://localhost:8080/tienda1/publico/pagar.jsp?modo=insertar&precio=2672&b1=pasar+por+caja', 'post http://localhost:8080/tienda1/publico/pagar.jsp', 'get http://localhost:8080/tiend

In [3]:
# prepare the dataset of input to output pairs encoded as integers
dataX = []
dataY = []
max_length=0
for data in dataset:
    max_length = max(max_length, len(data) - 1)

for data in dataset:
    seq_in = data[:len(data)-1]
    seq_out = data[1:]
    
    padding = [-n_vocab]*(max_length-len(seq_in))
    seq_encoding = [data_map[data_char] for data_char in seq_in]
    input_vector = seq_encoding + padding

    dataX.append(input_vector)
    dataY.append([data_map[data_char] for data_char in seq_out])

n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)


Total Patterns:  36000


In [ ]:
import numpy
from keras.utils import np_utils

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, max_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
temp_y = []
max_timesteps = max_length
for values in dataY:
    one_hot_encoding = []
    for value in values:
        temp = [0.0 for _ in range(n_vocab)]
        temp[value] = 1.0
        one_hot_encoding.append(temp)
    temp = [0.0 for _ in range(n_vocab)]
    for _ in range(361 - len(one_hot_encoding)):
        one_hot_encoding.append(temp)
    temp_y.append(one_hot_encoding)


Using TensorFlow backend.


In [ ]:
temp_y = numpy.reshape(temp_y, (n_patterns, max_length, n_vocab))
print(temp_y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Masking
from keras.layers import Dense
from keras.layers import TimeDistributed

model = Sequential()
model.add(Masking(mask_value = -1.0, input_shape = (max_length, 1)))
model.add(LSTM(50, input_shape = (max_length, 1), return_sequences = True))

model.add(TimeDistributed(Dense(y[0].shape[1], activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=2, batch_size=10, verbose=2)